In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
import copy

import re
from matplotlib import cm
import matplotlib.patches as patches

import os
from Bio import SeqIO
from Bio import Entrez

import scipy.stats
import math

In [2]:
sequence_df = pd.read_csv('preprocess_datasets/preprocessed_datasets/2019_guo_nci60_formatted_peptide_quants.tsv',
                          sep = '\t', index_col = 0)
sequence_df

,Peptide,Protein,gene symbol,R2,best mscore,numNA,Charge 1,Charge 2,Charge 3,Charge 4,...,RE_CAKI1_a,RE_CAKI1_b,RE_RXF393_a,RE_RXF393_b,RE_SN12C_a,RE_SN12C_b,RE_TK10_a,RE_TK10_b,RE_UO31_a,RE_UO31_b
1,VHCCFYFISPFGHGLKPLDVAFMK,Q15019,2-Sep,0.935026,1.920000e-07,4,0,0,0,0,...,184888.7,157024.0,49126.2,52422.3,37285.8,38880.8,179390.0,148828.3,208464.9,183507.2
3,VNIVPVIAK,Q15019,2-Sep,0.917310,4.010000e-09,0,0,1,0,0,...,623419.3,676515.0,521146.7,505380.1,512520.5,588922.3,836327.0,855696.7,844485.0,958940.3
4,QQPTQFINPETPGYVGFANLPNQVHR,Q15019,2-Sep,0.897582,2.160000e-08,0,0,0,1,0,...,121129.1,111926.1,82168.3,79697.7,86637.4,86046.4,162744.6,140120.6,158263.5,147559.2
5,STLINSLFLTDLYPER,Q15019,2-Sep,0.893410,1.060000e-08,0,0,1,0,0,...,332466.8,303315.1,262901.7,219749.2,238151.9,240847.5,381630.9,323716.2,416258.4,358276.0
6,ASIPFSVVGSNQLIEAK,Q15019,2-Sep,0.882319,9.200000e-09,0,0,1,0,0,...,313338.2,307400.9,267945.1,253036.2,275804.3,244736.9,366880.1,350980.3,423942.9,372984.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22541,CSVCSEPIMPEPGRDETVR,Q15942,ZYX,0.876235,8.910000e-07,2,0,0,1,0,...,126396.9,115465.7,155045.8,150804.4,91421.7,114228.9,112403.4,117977.3,141709.4,143503.7
22543,QHPVPPPAQNQNQVR,Q15942,ZYX,0.848883,7.970000e-05,14,0,1,0,0,...,9361.0,7864.9,15023.6,16419.2,8675.6,9419.2,8076.9,7327.0,14040.5,13399.5
22544,ALGQLFHIACFTCHQCAQQLQGQQFYSLEGAPYCEGCYTDTLEK,Q15942,ZYX,0.813338,7.390000e-08,2,0,0,0,0,...,72928.2,60348.4,92661.4,76823.6,53861.6,70917.5,72388.0,67317.0,79469.1,86790.7
22545,VSSGYVPPPVATPFSSK,Q15942,ZYX,0.649288,1.230000e-07,12,0,1,0,0,...,233653.1,323083.7,497715.0,479244.1,274234.3,235873.7,332520.8,247520.0,379462.9,321573.3


In [3]:
all_proteins = sequence_df['Protein'].values
all_unique_proteins = np.unique(sequence_df['Protein'].values)
print("All proteins: ", len(all_unique_proteins))
print("All proteins: ", all_unique_proteins)

all_peptide_sequences = sequence_df['Peptide']

print("All sequences: ", len(all_peptide_sequences))

All proteins:  2122
All proteins:  ['A0AV96' 'A0AVT1' 'A0MZ66' ... 'Q9Y6M5' 'Q9Y6M7' 'Q9Y6N5']
All sequences:  14472


## Download all the protein sequences

In [4]:
#Record protein sequences
all_protein_sequences = []
all_protein_information = []

for protein_id in all_unique_proteins:
    
    if not os.path.exists('protein_fasta_files/'):
        os.makedirs('protein_fasta_files/')
    
    filename = "protein_fasta_files/" + protein_id + ".fasta"
    if not os.path.isfile(filename):
        net_handle = Entrez.efetch(
            db="protein", id=protein_id, rettype="fasta", retmode="text"
        )
        out_handle = open(filename, "w")
        out_handle.write(net_handle.read())
        out_handle.close()
        net_handle.close()
        #print("Saved")

    #print("Parsing...")
    record = SeqIO.read(filename, "fasta")
    #print(record)
    all_protein_sequences.append(record.seq)
    all_protein_information.append(record)

## Match peptide sequences with proteins

In [5]:
#Record the indices for each peptide
peptide_sequence_start_indices = np.zeros(len(all_peptide_sequences))
peptide_sequence_end_indices = np.zeros(len(all_peptide_sequences))

for p_index, protein in enumerate(all_unique_proteins):
    
    protein_sequence = all_protein_sequences[p_index]
    print("Protein sequence: ", protein)
    print("Protein sequence: ", protein_sequence)
    
    peptides = all_peptide_sequences.iloc[np.where(all_proteins == protein)[0]]
    print(peptides)
    for peptide_index, peptide in enumerate(peptides):
        
        print("Peptide sequence:", peptide)
        print("Peptide sequence:", peptide_index)
        try:
            index_order = protein_sequence.index(peptide)
            print("Index of peptide:", index_order)
        except:
            print("Not detected")
        
        #Record the results
        print([np.where(sequence_df.index == peptides.index[peptide_index])[0]])
        peptide_sequence_start_indices[np.where(sequence_df.index == peptides.index[peptide_index])[0]] = index_order
        peptide_sequence_end_indices[np.where(sequence_df.index == peptides.index[peptide_index])[0]] = index_order + len(peptide)
        
        

Protein sequence:  A0AV96
Protein sequence:  MTAEDSTAAMSSDSAAGSSAKVPEGVAGAPNEAALLALMERTGYSMVQENGQRKYGGPPPGWEGPHPQRGCEVFVGKIPRDVYEDELVPVFEAVGRIYELRLMMDFDGKNRGYAFVMYCHKHEAKRAVRELNNYEIRPGRLLGVCCSVDNCRLFIGGIPKMKKREEILEEIAKVTEGVLDVIVYASAADKMKNRGFAFVEYESHRAAAMARRKLMPGRIQLWGHQIAVDWAEPEIDVDEDVMETVKILYVRNLMIETTEDTIKKSFGQFNPGCVERVKKIRDYAFVHFTSREDAVHAMNNLNGTELEGSCLEVTLAKPVDKEQYSRYQKAARGGGAAEAAQQPSYVYSCDPYTLAYYGYPYNALIGPNRDYFVKAGSIRGRGRGAAGNRAPGPRGSYLGGYSAGRGIYSRYHEGKGKQQEKGYELVPNLEIPTVNPVAIKPGTVAIPAIGAQYSMFPAAPAPKMIEDGKIHTVEHMISPIAVQPDPASAAAAAAAAAAAAAAVIPTVSTPPPFQGRPITPVYTVAPNVQRIPTAGIYGASYVPFAAPATATIATLQKNAAAAAAMYGGYAGYIPQAFPAAAIQVPIPDVYQTY
17067           VPEGVAGAPNEAALLALMER
17068    IPTAGIYGASYVPFAAPATATIATLQK
Name: Peptide, dtype: object
Peptide sequence: VPEGVAGAPNEAALLALMER
Peptide sequence: 0
Index of peptide: 21
[array([11065])]
Peptide sequence: IPTAGIYGASYVPFAAPATATIATLQK
Peptide sequence: 1
Index of peptide: 530
[array([11066])]
Protein sequence:  A0AVT1
Protein sequence:  MEGSEPVAAHQ

[array([8299])]
Peptide sequence: NPIMSQCFDR
Peptide sequence: 10
Index of peptide: 578
[array([8300])]
Peptide sequence: YLGLLENLR
Peptide sequence: 11
Index of peptide: 647
[array([8301])]
Peptide sequence: CPENAFFLDHVR
Peptide sequence: 12
Index of peptide: 801
[array([8302])]
Peptide sequence: LLSVEGSTLR
Peptide sequence: 13
Index of peptide: 340
[array([8303])]
Peptide sequence: NFHIFYQLLEGGEEETLRR
Peptide sequence: 14
Index of peptide: 235
[array([8304])]
Peptide sequence: NPQSYLYLVK
Peptide sequence: 15
Index of peptide: 259
[array([8305])]
Peptide sequence: QLLLTPNAVVIVEDAK
Peptide sequence: 16
Index of peptide: 938
[array([8306])]
Peptide sequence: GEELLSPLNLEQAAYAR
Peptide sequence: 17
Index of peptide: 361
[array([8307])]
Peptide sequence: GVSFYEVPPHLFAVADTVYR
Peptide sequence: 18
Index of peptide: 105
[array([8308])]
Protein sequence:  O00170
Protein sequence:  MADIIARLREDGIQKRVIQEGRGELPDFQDGTKATFHYRTLHSDDEGTVLDDSRARGKPMELIIGKKFKLPVWETIVCTMREGEIAQFLCDIKHVVLYPLVAKSLRNIAVGKDP

[array([6725])]
Peptide sequence: STACQMLVCYAK
Peptide sequence: 5
Index of peptide: 678
[array([6726])]
Peptide sequence: TIECISLIGLAVGK
Peptide sequence: 6
Index of peptide: 556
[array([6727])]
Peptide sequence: LCGDTSLNNMQR
Peptide sequence: 7
Index of peptide: 264
[array([6728])]
Peptide sequence: ITFLLQAIR
Peptide sequence: 8
Index of peptide: 41
[array([6729])]
Peptide sequence: QAEETYENIPGQSK
Peptide sequence: 9
Index of peptide: 27
[array([6730])]
Peptide sequence: VSDILHSIFSSYK
Peptide sequence: 10
Index of peptide: 841
[array([6731])]
Peptide sequence: VIAALLQTMEDQGNQR
Peptide sequence: 11
Index of peptide: 441
[array([6732])]
Peptide sequence: FMQDASDVMQLLLK
Peptide sequence: 12
Index of peptide: 572
[array([6733])]
Peptide sequence: VRGPEYLTQMWHFMCDALIK
Peptide sequence: 13
Index of peptide: 735
[array([6734])]
Peptide sequence: FMQDASDVMQLLLK
Peptide sequence: 14
Index of peptide: 572
[array([6735])]
Peptide sequence: QAAAYGLGVMAQYGGDNYRPFCTEALPLLVR
Peptide sequence: 15
In

[array([6989])]
Peptide sequence: DSQVVQVVLDGLK
Peptide sequence: 4
Index of peptide: 423
[array([6990])]
Protein sequence:  O00515
Protein sequence:  MAVSRKDWSALSSLARQRTLEDEEEQERERRRRHRNLSSTTDDEAPRLSQNGDRQASASERLPSVEEAEVPKPLPPASKDEDEDIQSILRTRQERRQRRQVVEAAQAPIQERLEAEEGRNSLSPVQATQKPLVSKKELEIPPRRRLSREQRGPWALEEESLVGREPEERKKGVPEKSPVLEKSSMPKKTAPEKSLVSDKTSISEKVLASEKTSLSEKIAVSEKRNSSEKKSVLEKTSVSEKSLAPGMALGSGRRLVSEKASIFEKALASEKSPTADAKPAPKRATASEQPLAQEPPASGGSPATTKEQRGRALPGKNLPSLAKQGASDPPTVASRLPPVTLQVKIPSKEEEADMSSPTQRTYSSSLKRSSPRTISFRMKPKKENSETTLTRSASMKLPDNTVKLGEKLERYHTAIRRSESVKSRGLPCTELFVAPVGVASKRHLFEKELAGQSRAEPASSRKENLRLSGVVTSRLNLWISRTQESGDQDPQEAQKASSATERTQWGQKSDSSLDAEV
11125          NLSSTTDDEAPR
11126         QVVEAAQAPIQER
11127    GPWALEEESLVGREPEER
11128     GLPCTELFVAPVGVASK
Name: Peptide, dtype: object
Peptide sequence: NLSSTTDDEAPR
Peptide sequence: 0
Index of peptide: 35
[array([7150])]
Peptide sequence: QVVEAAQAPIQER
Peptide sequence: 1
Index of peptide: 99
[array([7151])]
Peptide seque

[array([10305])]
Peptide sequence: EGQTICVR
Peptide sequence: 4
Index of peptide: 593
[array([10306])]
Peptide sequence: GPLVNASLR
Peptide sequence: 5
Index of peptide: 368
[array([10307])]
Peptide sequence: DLNCVPEIADTLGAVAK
Peptide sequence: 6
Index of peptide: 18
[array([10308])]
Protein sequence:  O14745
Protein sequence:  MSADAAAGAPLPRLCCLEKGPNGYGFHLHGEKGKLGQYIRLVEPGSPAEKAGLLAGDRLVEVNGENVEKETHQQVVSRIRAALNAVRLLVVDPETDEQLQKLGVQVREELLRAQEAPGQAEPPAAAEVQGAGNENEPREADKSHPEQRELRPRLCTMKKGPSGYGFNLHSDKSKPGQFIRSVDPDSPAEASGLRAQDRIVEVNGVCMEGKQHGDVVSAIRAGGDETKLLVVDRETDEFFKKCRVIPSQEHLNGPLPVPFTNGEIQKENSREALAEAALESPRPALVRSASSDTSEELNSQDSPPKQDSTAPSSTSSSDPILDFNISLAMAKERAHQKRSSKRAPQMDWSKKNELFSNL
18951                LLVVDPETDEQLQK
18952                SVDPDSPAEASGLR
18953    AQEAPGQAEPPAAAEVQGAGNENEPR
18954             EALAEAALESPRPALVR
18955                    LVEPGSPAEK
18956                  IVEVNGVCMEGK
18957                LLVVDRETDEFFKK
18960       VIPSQEHLNGPLPVPFTNGEIQK
Name: Peptide, dtype: ob

...

[array([12767])]
Peptide sequence: QTPLTCSGHTRPVVDLAFSGITPYGYFLISACK
Peptide sequence: 4
Index of peptide: 4
[array([12768])]
Peptide sequence: IYDLNKPEAEPK
Peptide sequence: 5
Index of peptide: 125
[array([12769])]
Peptide sequence: FSPDGELYASGSEDGTLR
Peptide sequence: 6
Index of peptide: 272
[array([12770])]
Peptide sequence: AATAAADFTAK
Peptide sequence: 7
Index of peptide: 73
[array([12771])]
Peptide sequence: EFLVAGGEDFK
Peptide sequence: 8
Index of peptide: 235
[array([12772])]
Peptide sequence: SIAFHSAVSLDPIK
Peptide sequence: 9
Index of peptide: 204
[array([12773])]
Peptide sequence: SFEAPATINSASLHPEK
Peptide sequence: 10
Index of peptide: 218
[array([12774])]
Peptide sequence: VWDAVSGDELMTLAHK
Peptide sequence: 11
Index of peptide: 84
[array([12775])]
Protein sequence:  Q9Y3I0
Protein sequence:  MSRSYNDELQFLEKINKNCWRIKKGFVPNMQVEGVFYVNDALEKLMFEELRNACRGGGVGGFLPAMKQIGNVAALPGIVHRSIGLPDVHSGYGFAIGNMAAFDMNDPEAVVSPGGVGFDINCGVRLLRTNLDESDVQPVKEQLAQAMFDHIPVGVGSKGVIPMNAKDLEEALEMGVDWSLREGY

[array([13239])]
Peptide sequence: AQEACGPLEMDSALSVVQNLEK
Peptide sequence: 51
Index of peptide: 1040
[array([13240])]
Peptide sequence: STVLQQQYNR
Peptide sequence: 52
Index of peptide: 428
[array([13241])]
Peptide sequence: ADAEGESDLENSRK
Peptide sequence: 53
Index of peptide: 841
[array([13242])]
Peptide sequence: AVTDSINQLITMCTQQAPGQK
Peptide sequence: 54
Index of peptide: 1340
[array([13243])]
Peptide sequence: LNEAAAGLNQAATELVQASR
Peptide sequence: 55
Index of peptide: 1241
[array([13244])]
Peptide sequence: VGAIPANALDDGQWSQGLISAAR
Peptide sequence: 56
Index of peptide: 2375
[array([13245])]
Peptide sequence: TMLESAGGLIQTAR
Peptide sequence: 57
Index of peptide: 1604
[array([13246])]
Peptide sequence: AAAFEEQENETVVVK
Peptide sequence: 58
Index of peptide: 2476
[array([13247])]
Peptide sequence: DPPSWSVLAGHSR
Peptide sequence: 59
Index of peptide: 1625
[array([13248])]
Peptide sequence: LASEAKPAAVAAENEEIGSHIK
Peptide sequence: 60
Index of peptide: 1895
[array([13249])]
Peptide seq

[array([12949])]
Protein sequence:  Q9Y4W2
Protein sequence:  MSWESGAGPGLGSQGMDLVWSAWYGKCVKGKGSLPLSAHGIVVAWLSRAEWDQVTVYLFCDDHKLQRYALNRITVWRSRSGNELPLAVASTADLIRCKLLDVTGGLGTDELRLLYGMALVRFVNLISERKTKFAKVPLKCLAQEVNIPDWIVDLRHELTHKKMPHINDCRRGCYFVLDWLQKTYWCRQLENSLRETWELEEFREGIEEEDQEEDKNIVVDDITEQKPEPQDDGKSTESDVKADGDSKGSEEVDSHCKKALSHKELYERARELLVSYEEEQFTVLEKFRYLPKAIKAWNNPSPRVECVLAELKGVTCENREAVLDAFLDDGFLVPTFEQLAALQIEYEDGQTEVQRGEGTDPKSHKNVDLNDVLVPKPFSQFWQPLLRGLHSQNFTQALLERMLSELPALGISGIRPTYILRWTVELIVANTKTGRNARRFSAGQWEARRGWRLFNCSASLDWPRMVESCLGSPCWASPQLLRIIFKAMGQGLPDEEQEKLLRICSIYTQSGENSLVQEGSEASPIGKSPYTLDSLYWSVKPASSSFGSEAKAQQQEEQGSVNDVKEEEKEEKEVLPDQVEEEEENDDQEEEEEDEDDEDDEEEDRMEVGPFSTGQESPTAENARLLAQKRGALQGSAWQVSSEDVRWDTFPLGRMPGQTEDPAELMLENYDTMYLLDQPVLEQRLEPSTCKTDTLGLSCGVGSGNCSNSSSSNFEGLLWSQGQLHGLKTGLQLF
11245    MLSELPALGISGIRPTYILR
11246           AMGQGLPDEEQEK
Name: Peptide, dtype: object
Peptide sequence: MLSELPALGISGIRPTYILR
Peptide sequence: 0
Index of peptide: 401
[array([7249])]
Peptide sequence:

[array([8835])]
Peptide sequence: TLEEGHDFIQEFPGSPAFAALTSIAQK
Peptide sequence: 1
Index of peptide: 234
[array([8836])]
Protein sequence:  Q9Y5Z4
Protein sequence:  MAEPLQPDPGAAEDAAAQAVETPGWKAPEDAGPQPGSYEIRHYGPAKWVSTSVESMDWDSAIQTGFTKLNSYIQGKNEKEMKIKMTAPVTSYVEPGSGPFSESTITISLYIPSEQQFDPPRPLESDVFIEDRAEMTVFVRSFDGFSSAQKNQEQLLTLASILREDGKVFDEKVYYTAGYNSPVKLLNRNNEVWLIQKNEPTKENE
8632    NQEQLLTLASILR
8633     VYYTAGYNSPVK
Name: Peptide, dtype: object
Peptide sequence: NQEQLLTLASILR
Peptide sequence: 0
Index of peptide: 150
[array([5879])]
Peptide sequence: VYYTAGYNSPVK
Peptide sequence: 1
Index of peptide: 172
[array([5880])]
Protein sequence:  Q9Y617
Protein sequence:  MDAPRQVVNFGPGPAKLPHSVLLEIQKELLDYKGVGISVLEMSHRSSDFAKIINNTENLVRELLAVPDNYKVIFLQGGGCGQFSAVPLNLIGLKAGRCADYVVTGAWSAKAAEEAKKFGTINIVHPKLGSYTKIPDPSTWNLNPDASYVYYCANETVHGVEFDFIPDVKGAVLVCDMSSNFLSKPVDVSKFGVIFAGAQKNVGSAGVTVVIVRDDLLGFALRECPSVLEYKVQAGNSSLYNTPPCFSIYVMGLVLEWIKNNGGAAAMEKLSSIKSQTIYEIIDNSQGFYVCPVEPQNRSKMNIPFRIGNAKGDDALEKRFLDKALELNMLSL

[array([12134])]
Peptide sequence: ESALILLQTVPK
Peptide sequence: 2
Index of peptide: 337
[array([12135])]
Protein sequence:  Q9Y6M7
Protein sequence:  MERFRLEKKLPGPDEEAVVDLGKTSSTVNTKFEKEELESHRAVYIGVHVPFSKESRRRHRHRGHKHHHRRRKDKESDKEDGRESPSYDTPSQRVQFILGTEDDDEEHIPHDLFTEMDELCYRDGEEYEWKETARWLKFEEDVEDGGDRWSKPYVATLSLHSLFELRSCILNGTVMLDMRASTLDEIADMVLDNMIASGQLDESIRENVREALLKRHHHQNEKRFTSRIPLVRSFADIGKKHSDPHLLERNGEGLSASRHSLRTGLSASNLSLRGESPLSLLLGHLLPSSRAGTPAGSRCTTPVPTPQNSPPSSPSISRLTSRSSQESQRQAPELLVSPASDDIPTVVIHPPEEDLEAALKGEEQKNEENVDLTPGILASPQSAPGNLDNSKSGEIKGNGSGGSRENSTVDFSKVDMNFMRKIPTGAEASNVLVGEVDFLERPIIAFVRLAPAVLLTGLTEVPVPTRFLFLLLGPAGKAPQYHEIGRSIATLMTDEIFHDVAYKAKDRNDLLSGIDEFLDQVTVLPPGEWDPSIRIEPPKSVPSQEKRKIPVFHNGSTPTLGETPKEAAHHAGPELQRTGRLFGGLILDIKRKAPFFLSDFKDALSLQCLASILFLYCACMSPVITFGGLLGEATEGRISAIESLFGASLTGIAYSLFAGQPLTILGSTGPVLVFEKILYKFCRDYQLSYLSLRTSIGLWTSFLCIVLVATDASSLVCYITRFTEEAFAALICIIFIYEALEKLFDLGETYAFNMHNNLDKLTSYSCVCTEPPNPSNETLAQWKKDNITAHNISWRNLTVSECKKLRGVFLGSACGHHGPYIPDVLFWCVILFFTTFFLSSFLKQFKTK

In [6]:
sequence_df['Peptide start index'] = peptide_sequence_start_indices
sequence_df['Peptide end index'] = peptide_sequence_end_indices

#Record results
peptide_indices = sequence_df[['Peptide', 'Protein', 'gene symbol', 'R2', 'best mscore',
       'numNA', 'Charge 2', 'Charge 3', 'Charge 4', 'Charge 5', 'Charge 6', 
                          'Peptide start index', 'Peptide end index']]

peptide_indices.index = sequence_df.index
peptide_indices

,Peptide,Protein,gene symbol,R2,best mscore,numNA,Charge 2,Charge 3,Charge 4,Charge 5,Charge 6,Peptide start index,Peptide end index
1,VHCCFYFISPFGHGLKPLDVAFMK,Q15019,2-Sep,0.935026,1.920000e-07,4,0,0,0,1,0,145.0,169.0
3,VNIVPVIAK,Q15019,2-Sep,0.917310,4.010000e-09,0,1,0,0,0,0,174.0,183.0
4,QQPTQFINPETPGYVGFANLPNQVHR,Q15019,2-Sep,0.897582,2.160000e-08,0,0,1,0,0,0,3.0,29.0
5,STLINSLFLTDLYPER,Q15019,2-Sep,0.893410,1.060000e-08,0,1,0,0,0,0,50.0,66.0
6,ASIPFSVVGSNQLIEAK,Q15019,2-Sep,0.882319,9.200000e-09,0,1,0,0,0,0,232.0,249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22541,CSVCSEPIMPEPGRDETVR,Q15942,ZYX,0.876235,8.910000e-07,2,0,1,0,0,0,503.0,522.0
22543,QHPVPPPAQNQNQVR,Q15942,ZYX,0.848883,7.970000e-05,14,1,0,0,0,0,328.0,343.0
22544,ALGQLFHIACFTCHQCAQQLQGQQFYSLEGAPYCEGCYTDTLEK,Q15942,ZYX,0.813338,7.390000e-08,2,0,0,0,1,0,399.0,443.0
22545,VSSGYVPPPVATPFSSK,Q15942,ZYX,0.649288,1.230000e-07,12,1,0,0,0,0,167.0,184.0


## Read abundance matrices

In [7]:
output_filename = 'trained_models/2019_guo_nci60/2019_guo_nci60_'

#Read Q, alpha, and c matrices
Q_train = pd.read_csv(output_filename + 'Q_train.tsv', sep = '\t', index_col = 0)
Q_test = pd.read_csv(output_filename + 'Q_test.tsv', sep = '\t', index_col = 0)

P_train = pd.read_csv(output_filename + 'P_train.tsv', sep = '\t', index_col = 0)
P_test = pd.read_csv(output_filename + 'P_test.tsv', sep = '\t', index_col = 0)

predicted_train_coeffs = pd.read_csv(output_filename + 'coefficients_train.tsv', sep = '\t', index_col = 0)
predicted_test_coeffs = pd.read_csv(output_filename + 'coefficients_test.tsv', sep = '\t', index_col = 0)
predicted_train_coeffs.index = Q_train.index
predicted_test_coeffs.index = Q_test.index



FileNotFoundError: [Errno 2] No such file or directory: 'trained_models/2019_guo_nci60/2019_guo_nci60_Q_train.tsv'

## Create plots

In [ ]:
def createJointPlots(protein_name):
    
    ############################################
    # Define plot shapes
    SMALL_SIZE = 60
    MEDIUM_SIZE = 60
    BIGGER_SIZE = 90

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex = True, figsize=(40, 30))
    fig.tight_layout()
    
    
    ############################################
    #We want to sort the peptides and then get the abundances
    
    sorted_peptide_indices = peptide_indices.loc[sequence_df[sequence_df['Protein'] == protein_name].index].sort_values(by = ['Peptide start index', 'Peptide end index']).index
   
    #Get sorted raw abundances
    index_values = peptide_indices.loc[sorted_peptide_indices]['Peptide start index'].values
    index_values_end = peptide_indices.loc[sorted_peptide_indices]['Peptide end index'].values
    raw_peptide_abundances = Q_test.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices]

    mean_raw_abundance = np.mean(raw_peptide_abundances.values)
    print("Mean raw abundance: ", mean_raw_abundance)
    
    #Plot the mean abundance across all runs for each peptide
    raw_peptide_abundances_run_mean = np.mean(raw_peptide_abundances.values, axis = 1)
   

    #Obtain adjusted abundances using coefficients
    sorted_peptide_indices = peptide_indices.loc[sequence_df[sequence_df['Protein'] == protein_name].index].sort_values(by = ['Peptide start index', 'Peptide end index']).index
    raw_peptide_abundances = Q_test.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices]

    coeffs = predicted_test_coeffs.loc[sequence_df[sequence_df['Protein'] == protein_name].index].loc[sorted_peptide_indices].values.ravel().reshape((-1, 1))
    adjusted_peptide_abundances = raw_peptide_abundances / np.abs(coeffs)
    
    mean_adjusted_abundance = np.mean(adjusted_peptide_abundances.values)
    print("Mean adjusted abundance: ", mean_adjusted_abundance)
    
    #Plot the mean abundance across all runs for each peptide
    adjusted_peptide_abundances_run_mean = np.mean(adjusted_peptide_abundances.values, axis = 1)
    
    
    ############################################
    #Center all the raw and observed abundances
    
    #Calculate CV before centering
    raw_CV = np.std(raw_peptide_abundances_run_mean) / np.mean(raw_peptide_abundances_run_mean)
    adj_CV = np.std(adjusted_peptide_abundances_run_mean) / np.mean(adjusted_peptide_abundances_run_mean)
    
    raw_peptide_abundances_run_mean = raw_peptide_abundances_run_mean - mean_raw_abundance
    adjusted_peptide_abundances_run_mean = adjusted_peptide_abundances_run_mean - mean_adjusted_abundance
    
    mean_raw_abundance = np.mean(raw_peptide_abundances_run_mean)
    mean_adjusted_abundance = np.mean(adjusted_peptide_abundances_run_mean)
    
    
    ############################################
    #Plot the raw peptide abundances
    
    #Plot abundances of all peptides
    for i in range(len(raw_peptide_abundances_run_mean)):
        peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]
          
        ax1.vlines(x = (index_values[i] + index_values_end[i]) / 2, 
                       ymin = mean_raw_abundance, ymax = raw_peptide_abundances_run_mean[i],
                   linewidth = 5, color = '#eb4d4b')
        
        ax1.scatter((index_values[i] + index_values_end[i]) / 2, raw_peptide_abundances_run_mean[i], alpha = 0.5, 
                    s = 1500, color = '#eb4d4b', zorder = 10)
        
    protein_sequence = all_protein_sequences[np.where(all_unique_proteins == protein_name)[0][0]]
    
    #Plot the mean raw peptide abundance
    ax1.plot(np.arange(len(protein_sequence)), 
              mean_raw_abundance * np.ones(len(protein_sequence)), 
              lw = 5, color = 'Black', label = 'Mean observed abundance')

    
    ############################################
    #Plot the adjusted peptide abundances
    
    for i in range(len(adjusted_peptide_abundances_run_mean)):
        peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]
        
        ax2.vlines((index_values[i] + index_values_end[i]) / 2, 
                   ymin = mean_adjusted_abundance, ymax = adjusted_peptide_abundances_run_mean[i],
               linewidth = 5, color = '#8854d0')
        ax2.scatter((index_values[i] + index_values_end[i]) / 2, 
                    adjusted_peptide_abundances_run_mean[i], alpha = 0.5, 
                s = 1500, color = '#8854d0', zorder = 10)
        
    ax2.plot(np.arange(len(protein_sequence)), 
              mean_adjusted_abundance * np.ones(len(protein_sequence)), 
              lw = 5, color = 'Black', label = 'Mean adjusted abundance')

    
    ############################################
    #Plot the alignment of peptides along the protein, colored by charge 
    
    current_height = 0
    for charge in range(2, 7):
        
        #Plot each peptide as a bar plot
        current_end_indices = [0]
        available_heights = [0]

        for i in range(len(raw_peptide_abundances_run_mean)):
            peptide_charges = peptide_indices.loc[sorted_peptide_indices].iloc[i]

            if peptide_charges['Charge 2'] == 1:
                color_value = '#22a6b3'
            elif peptide_charges['Charge 3'] == 1:
                color_value = '#f0932b'
            elif peptide_charges['Charge 4'] == 1:
                color_value = '#f9ca24'
            elif peptide_charges['Charge 5'] == 1:
                color_value = '#be2edd'
            elif peptide_charges['Charge 6'] == 1:
                color_value = '#badc58'

            if peptide_charges['Charge ' + str(charge)] == 1:
                start_of_peptide = index_values[i]
                end_of_peptide = index_values_end[i]

                #Compare the start site to all heights
                plotted_flag = False
                for height in available_heights:
                    if not plotted_flag and start_of_peptide > current_end_indices[height]:
        
                        rect = patches.Rectangle((index_values[i], current_height + (height/2)), 
                                      index_values_end[i] - index_values[i], 0.4,
                                      facecolor = color_value, linewidth=2, 
                                      alpha = 0.5, edgecolor='black')
                        current_end_index = end_of_peptide
                        current_end_indices[height] = current_end_index
                        plotted_flag = True
                        ax3.add_patch(rect)
                    
                height = available_heights[-1]
                if not plotted_flag:
                    available_heights.append((height + 1))
                    height = height + 1
                    rect = patches.Rectangle((index_values[i], current_height + (height/2)), 
                                      index_values_end[i] - index_values[i], 0.4,
                                      facecolor = color_value, linewidth=2, 
                                      alpha = 0.5, edgecolor='black')
                    ax3.add_patch(rect)
                    current_end_indices.append(end_of_peptide)

            
        #Increase height 
        current_height = current_height + (available_heights[-1]/2) + 0.5
        
        
    ############################################
    #Plot cleavage points

    protein_sequence = all_protein_sequences[np.where(all_unique_proteins == protein_name)[0][0]]
    for i in range(len(protein_sequence)-1):
        if protein_sequence[i:i+1] == 'K' or protein_sequence[i:i+1] == 'R':
            
            ax3.vlines(x = i+1, 
                      ymin = -0.1, 
                      ymax = 0.0,
                      linewidth = 2,
                      color = 'black')
    
    
    ############################################
    #Define grids, labels, axes
    
    ax1.grid('minor')
    ax2.grid('minor')
    
    max_value = max(np.nanmax(raw_peptide_abundances_run_mean), 
                    np.nanmax(adjusted_peptide_abundances_run_mean)) * 1.25
    
    min_value = min(np.nanmin(raw_peptide_abundances_run_mean), 
                    np.nanmin(adjusted_peptide_abundances_run_mean)) * 1.5
    
    ax1.set_ylim([min_value, max_value])
    ax2.set_ylim([min_value, max_value])
    
    
    ax3.set_xlabel('Peptide index of protein ' + protein_name)
    ax1.set_ylabel('Centered \n observed \n abundances')
    ax2.set_ylabel('Centered \n adjusted \n abundances')
    ax3.set_ylabel('Peptides \n along \n the protein')
    
    raw_error = np.mean(np.abs(raw_peptide_abundances_run_mean - mean_raw_abundance)**2)
    adj_error = np.mean(np.abs(adjusted_peptide_abundances_run_mean - mean_adjusted_abundance)**2)
    percent_improvement = ((raw_error - adj_error) / raw_error) * 100
    
    #Count no of peptides that becomes closer to the mean
    count_peptide = np.where(np.abs(adjusted_peptide_abundances_run_mean - mean_adjusted_abundance) < np.abs(raw_peptide_abundances_run_mean - mean_raw_abundance))[0]
    print("No of peptides with smaller error: ", len(count_peptide))
    print("Percent of peptides with smaller error: ", (len(count_peptide) / len(adjusted_peptide_abundances_run_mean)))
    
    print("Error for raw values: ", raw_error)
    print("Error for adjusted values: ", adj_error)
    print("Percent improvement: ", percent_improvement)
    
    percent_reduction = ((raw_CV - adj_CV) / raw_CV) * 100
    print("CV for raw values: ", raw_CV)
    print("CV for adjusted values: ", adj_CV)
    print("Percent reduction: ", percent_reduction)   
    
    plt.xlim([-5, len(protein_sequence) + 5]) 

    ax3.set_ylim([0, current_height])
    
    plt.show()

In [ ]:
#Select one of the proteins containing many peptides
from collections import Counter

#Select one of the proteins containing many peptides
protein_names = sequence_df.loc[np.intersect1d(Q_test.index, sequence_df.index)].groupby('Protein').count().sort_values(by = 'Peptide')[::-1][:40]
protein_names = protein_names.index
protein_names

In [ ]:
#Plots for top 10 test proteins
for i in range(10):
    createJointPlots(protein_names[i])
    